# Applied Data Mining -- Homework3
## name: Yimeng Zhao
## UT EID: yz9428
### (a) 


In [1]:
import warnings
warnings.filterwarnings('ignore')

import nltk 
from nltk import word_tokenize
from nltk.corpus import stopwords

import gensim # topic modeling
from gensim import corpora, models

In [2]:
# Merge together and treat each new-line as a single document
import os

# Open a file
dirpath = "../../downloads/merged/"
dirs = os.listdir(dirpath)
merged = []
slice_index = [] #to store where the file ends, for example, the first file,democratic file,
                            # is form 0 to 1807, so slick_index[0]=1808 

for file in dirs:
    fin = open(os.path.join(dirpath,file),'rU')
    raw = fin.readlines()
    #print (len(raw),type(raw))
    for line in raw:
        merged.append(line)
    slice_index.append(len(merged))
    fin.close()
print('instances:',len(merged))

instances: 3081


In [3]:
# Tokenize
import string


def tokenize_docs(doc_list):
    tokenized_docs = []
    count = 0 # count total words
    
    for doc in doc_list:
        '''# lowercase the words
        doc_lowercase = doc.lower()'''
    
        # now tokenize
        #doc_tokens = nltk.tokenize.word_tokenize(doc_lowercase)
        doc_tokens = gensim.utils.tokenize(doc,lowercase = True, deacc=True)
    
        # remove meaningless words
        stop_list = stopwords.words('english')
        stop_list.extend(string.punctuation)
        stop_list.extend(["''","...","mdash","/i","``","would","'m","'s","'re","'ve","'ll","n't",
                          "applause","laughter","crosstalk","--"])
    
        doc_tokens = [word for word in doc_tokens if not word in stop_list]
        tokenized_docs.append(doc_tokens)
        count = count+len(doc_tokens)
        
    print("after tokenizing and removing some meaningless words, there are ",count," words")
    return tokenized_docs


In [4]:
# Stem 
from nltk.stem.porter import PorterStemmer

def stem_docs(tokenized_docs):
    p_stemmer = PorterStemmer()
    tokenized_stemmed = [[p_stemmer.stem(w) for w in doc] for 
                             doc in tokenized_docs]
    
    # create a 'dictionary' for tokenzied and stemmed docs
    dictionary = corpora.Dictionary(tokenized_stemmed)
    print(dictionary)
    
    corpus = [dictionary.doc2bow(text) for text in tokenized_stemmed]
    
    return dictionary,corpus

In [5]:
toke_merged = tokenize_docs(merged)
dict_merged, corpus_merged = stem_docs(toke_merged)

after tokenizing and removing some meaningless words, there are  54314  words
Dictionary(4151 unique tokens: ['scar', 'gold', 'bag', 'incred', 'texa']...)




3081 instances. After tokenizing and stemming(remove many meaningless words), there are 54314 words and 4151 unique tokens. 

In [6]:
def topic_model(corpus,dictionary,topics,topwords):
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=topics, 
                                            id2word = dictionary, passes=10)
    print(lda_model.print_topics(num_topics=topics, num_words=topwords),'\n')
    

topics = [5,10,20]
topwords = 10
for k in topics:
    print(k,' topics:')
    topic_model(corpus_merged,dict_merged,k,topwords)
    

5  topics:
[(0, '0.014*isi + 0.011*need + 0.010*new + 0.010*countri + 0.009*presid + 0.009*state + 0.008*know + 0.008*make + 0.008*peopl + 0.007*well'), (1, '0.015*state + 0.010*one + 0.010*said + 0.010*talk + 0.009*want + 0.009*say + 0.009*unit + 0.009*clinton + 0.008*go + 0.008*hillari'), (2, '0.019*peopl + 0.014*think + 0.013*go + 0.013*countri + 0.012*presid + 0.012*need + 0.010*american + 0.009*work + 0.009*get + 0.008*know'), (3, '0.026*tax + 0.014*go + 0.012*right + 0.011*want + 0.011*pay + 0.011*let + 0.011*obama + 0.011*money + 0.010*barack + 0.010*countri'), (4, '0.025*peopl + 0.021*go + 0.013*know + 0.009*question + 0.009*get + 0.008*wall + 0.008*back + 0.008*let + 0.008*well + 0.007*countri')] 

10  topics:
[(0, '0.025*presid + 0.024*clinton + 0.021*countri + 0.019*hillari + 0.018*peopl + 0.015*elect + 0.015*america + 0.013*want + 0.013*obama + 0.012*state'), (1, '0.018*peopl + 0.014*said + 0.012*good + 0.009*think + 0.009*get + 0.009*problem + 0.008*democrat + 0.008*come +

k = 10 is the best of these three ks. The topics of k = 10 are more understandable than those of k = 5, for example, in k = 5, (0, '0.014*isi + 0.011*need + 0.010*new + 0.010*countri + 0.009*presid + 0.009*state + 0.008*know + 0.008*make + 0.008*peopl + 0.007*well'), it is not clear what it means: it may be related to ISIS but very blur; while in k = 10, (0, '0.025*presid + 0.024*clinton + 0.021*countri + 0.019*hillari + 0.018*peopl + 0.015*elect + 0.015*america + 0.013*want + 0.013*obama + 0.012*state'), it seems like about election or president. 
The topics of k = 10 are better than those of k = 20, because k = 20 seems to have more overlaps and some topic words doen't make any sense.

### (c)

In [7]:
print("democratic:")
toke_democ = tokenize_docs(merged[:slice_index[0]])
dict_democ, corpus_democ = stem_docs(toke_democ)

print("\n","republican:")
toke_repub = tokenize_docs(merged[slice_index[0]:])
dict_repub, corpus_repub = stem_docs(toke_repub)

democratic:
after tokenizing and removing some meaningless words, there are  31058  words
Dictionary(3126 unique tokens: ['scar', 'gold', 'incred', 'learn', 'took']...)

 republican:
after tokenizing and removing some meaningless words, there are  23256  words
Dictionary(2818 unique tokens: ['gold', 'incred', 'texa', 'learn', 'took']...)


In [8]:
# Democratic topics
topics = [5,10,20]
for k in topics:
    print(k,' topics:')
    topic_model(corpus_democ,dict_democ,k,topwords),'\n'

5  topics:
[(0, '0.016*think + 0.014*know + 0.013*well + 0.011*work + 0.010*go + 0.010*countri + 0.010*peopl + 0.007*want + 0.006*get + 0.006*senat'), (1, '0.014*peopl + 0.013*go + 0.011*colleg + 0.010*care + 0.010*make + 0.010*pay + 0.009*think + 0.009*health + 0.008*want + 0.008*way'), (2, '0.019*peopl + 0.016*gun + 0.014*state + 0.010*say + 0.009*right + 0.007*american + 0.007*vote + 0.007*america + 0.007*look + 0.006*one'), (3, '0.016*street + 0.015*let + 0.015*wall + 0.012*secretari + 0.010*go + 0.010*know + 0.009*talk + 0.008*bank + 0.008*state + 0.007*governor'), (4, '0.016*think + 0.015*need + 0.015*peopl + 0.011*get + 0.011*countri + 0.011*go + 0.009*make + 0.009*american + 0.009*got + 0.008*believ')] 

10  topics:
[(0, '0.018*need + 0.014*work + 0.011*new + 0.009*togeth + 0.009*state + 0.009*countri + 0.008*thing + 0.008*think + 0.007*get + 0.007*well'), (1, '0.019*think + 0.015*peopl + 0.015*presid + 0.015*right + 0.011*know + 0.010*want + 0.010*state + 0.009*well + 0.008*ne

In [9]:
# Republician topics
topics = [5,10,20]
for k in topics:
    print(k,' topics:')
    topic_model(corpus_repub,dict_repub,k,topwords),'\n'

5  topics:
[(0, '0.023*go + 0.013*peopl + 0.012*presid + 0.010*obama + 0.010*countri + 0.009*fight + 0.008*barack + 0.008*america + 0.008*state + 0.008*isi'), (1, '0.017*peopl + 0.015*get + 0.014*want + 0.011*think + 0.011*go + 0.010*one + 0.009*say + 0.007*presid + 0.007*let + 0.007*us'), (2, '0.019*tax + 0.009*one + 0.008*peopl + 0.008*want + 0.008*year + 0.008*well + 0.008*thing + 0.007*know + 0.007*new + 0.006*go'), (3, '0.018*go + 0.016*know + 0.013*peopl + 0.011*countri + 0.010*presid + 0.009*need + 0.008*state + 0.008*work + 0.007*isi + 0.007*get'), (4, '0.018*countri + 0.016*peopl + 0.014*presid + 0.012*go + 0.009*know + 0.008*issu + 0.008*law + 0.008*year + 0.007*come + 0.007*first')] 

10  topics:
[(0, '0.022*tax + 0.017*state + 0.016*want + 0.014*us + 0.012*america + 0.011*peopl + 0.011*unit + 0.009*busi + 0.009*countri + 0.009*presid'), (1, '0.021*presid + 0.014*like + 0.012*countri + 0.012*make + 0.011*elect + 0.011*american + 0.010*america + 0.010*need + 0.008*let + 0.007

In Democratic file, they might talk about sequrity(ISIS), work, health care, gun, tax, campaign, Republician, Hillary Clinton. Republicians talk about ISIS, election, China(compaign), tax, law or legality, immigration, military, Hillary Clinton. They both has some topics which less appear in merged file. Furthermore, there are some differences between these two parties. For example, in Democratic's speech, ISIS is often related with countries, United States, etc.; while related with fight, president in Republician's. 